In [9]:
import numpy as np
import pandas as pd
from lifelines import KaplanMeierFitter, CoxPHFitter
import matplotlib.pyplot as plt

In [ ]:
!pip install -U numpy

In [ ]:
!pip install -U pandas

In [ ]:
!pip install -U matplotlib

In [ ]:
!pip install -U lifelines

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!pip install lifelines

## Univariate Cox, Multivariate Cox

In [10]:
data = pd.read_csv("lung.csv")

In [11]:
data.head()

,inst,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,3.0,306,1,74,1,1.0,90.0,100.0,1175.0,NaN
1,3.0,455,1,68,1,0.0,90.0,90.0,1225.0,15.0
2,3.0,1010,0,56,1,0.0,90.0,90.0,NaN,15.0
3,5.0,210,1,57,1,1.0,90.0,60.0,1150.0,11.0
4,1.0,883,1,60,1,0.0,100.0,90.0,NaN,0.0


In [12]:
data.shape

(228, 10)

In [13]:
data = data[['time', 'status', 'age', 'sex', 'ph.ecog', 'ph.karno','pat.karno', 'meal.cal', 'wt.loss']]
data["sex"] = data["sex"] - 1
data.head()

,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,306,1,74,0,1.0,90.0,100.0,1175.0,NaN
1,455,1,68,0,0.0,90.0,90.0,1225.0,15.0
2,1010,0,56,0,0.0,90.0,90.0,NaN,15.0
3,210,1,57,0,1.0,90.0,60.0,1150.0,11.0
4,883,1,60,0,0.0,100.0,90.0,NaN,0.0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       228 non-null    int64  
 1   status     228 non-null    int64  
 2   age        228 non-null    int64  
 3   sex        228 non-null    int64  
 4   ph.ecog    227 non-null    float64
 5   ph.karno   227 non-null    float64
 6   pat.karno  225 non-null    float64
 7   meal.cal   181 non-null    float64
 8   wt.loss    214 non-null    float64
dtypes: float64(5), int64(4)
memory usage: 16.2 KB


In [15]:
data["ph.karno"].fillna(data["ph.karno"].mean(), inplace = True)
data["pat.karno"].fillna(data["pat.karno"].mean(), inplace = True)
data["meal.cal"].fillna(data["meal.cal"].mean(), inplace = True)
data["wt.loss"].fillna(data["wt.loss"].mean(), inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       228 non-null    int64  
 1   status     228 non-null    int64  
 2   age        228 non-null    int64  
 3   sex        228 non-null    int64  
 4   ph.ecog    227 non-null    float64
 5   ph.karno   228 non-null    float64
 6   pat.karno  228 non-null    float64
 7   meal.cal   228 non-null    float64
 8   wt.loss    228 non-null    float64
dtypes: float64(5), int64(4)
memory usage: 16.2 KB


In [16]:
data.dropna(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227 entries, 0 to 227
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       227 non-null    int64  
 1   status     227 non-null    int64  
 2   age        227 non-null    int64  
 3   sex        227 non-null    int64  
 4   ph.ecog    227 non-null    float64
 5   ph.karno   227 non-null    float64
 6   pat.karno  227 non-null    float64
 7   meal.cal   227 non-null    float64
 8   wt.loss    227 non-null    float64
dtypes: float64(5), int64(4)
memory usage: 17.7 KB


In [17]:
coxph = CoxPHFitter()
data_age = data[['time', 'status', 'age']]
coxph.fit(data_age, duration_col = 'time', event_col = 'status')
coxph.print_summary()

D:\Anaconda\envs\biostat\lib\site-packages\lifelines\utils\printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 227
number of events observed = 164
   partial log-likelihood = -742.32
         time fit was run = 2022-09-06 06:57:01 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.02       1.02       0.01             0.00             0.04                 1.00                 1.04

            cmp to    z    p   -log2(p)
covariate                              
age           0.00 2.05 0.04       4.65
---
Concordance = 0.55
Partial AIC = 1486.64
log-likelihood ratio test = 4.33 on 1 df
-log2(p) of ll-ratio test = 4.74

In [18]:
coxph.check_assumptions(data_age, p_value_threshold=0.05)

Proportional hazard assumption looks okay.


[]

In [19]:
coxph = CoxPHFitter()
coxph.fit(data, duration_col = 'time', event_col = 'status')


<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>

In [20]:
coxph.print_summary()

D:\Anaconda\envs\biostat\lib\site-packages\lifelines\utils\printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 227
number of events observed = 164
   partial log-likelihood = -725.64
         time fit was run = 2022-09-06 06:57:01 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.01       1.01       0.01            -0.01             0.03                 0.99                 1.03
sex        -0.59       0.56       0.17            -0.92            -0.25                 0.40                 0.78
ph.ecog     0.62       1.85       0.18             0.26             0.98                 1.29                 2.66
ph.karno    0.02       1.02       0.01            -0.00             0.03                 1.00                 1.03
pat.karno  -0.01       0.99       0.01            -0.03             0.00                 0.97                 1.00
meal.cal    0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
wt.loss    -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00

            cmp to     z      p   -log2(p)
covariate                                 
age           0.00  1.35   0.18       2.49
sex           0.00 -3.44 <0.005      10.76
ph.ecog       0.00  3.35 <0.005      10.28
ph.karno      0.00  1.66   0.10       3.36
pat.karno     0.00 -1.77   0.08       3.70
meal.cal      0.00  0.10   0.92       0.12
wt.loss       0.00 -1.73   0.08       3.57
---
Concordance = 0.65
Partial AIC = 1465.28
log-likelihood ratio test = 37.68 on 7 df
-log2(p) of ll-ratio test = 18.13

In [21]:
coxph.check_assumptions(data, p_value_threshold=0.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



D:\Anaconda\envs\biostat\lib\site-packages\lifelines\statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>
         test_name = proportional_hazard_test

---
                test_statistic    p  -log2(p)
age       km              0.33 0.57      0.82
          rank            0.12 0.72      0.47
meal.cal  km              5.46 0.02      5.68
          rank            4.88 0.03      5.20
pat.karno km              0.41 0.52      0.94
          rank            0.38 0.54      0.89
ph.ecog   km              1.25 0.26      1.93
          rank            0.94 0.33      1.59
ph.karno  km              3.52 0.06      4.04
          rank            2.98 0.08      3.57
sex       km              2.81 0.09      3.42
          rank            2.69 0.10      3.31
wt.loss   km              0.02 0.89      0.16
          rank            0.06 0.81      0.31



1. Variable 'meal.cal' failed the non-proportional test: p-value is 0.0194.

   Advice 1: the functional form of the variable 'meal.cal' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'meal.cal' using pd.cut, and then specify it in
`strata=['meal.cal', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#

[]

In [22]:
data_1 = data[['time', 'status', 'age', 'sex', 'ph.ecog', 'ph.karno','pat.karno', 'wt.loss']]

In [23]:
coxph.fit(data_1, duration_col = 'time', event_col = 'status')

<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>

In [24]:
coxph.print_summary()

D:\Anaconda\envs\biostat\lib\site-packages\lifelines\utils\printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 227 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 227
number of events observed = 164
   partial log-likelihood = -725.64
         time fit was run = 2022-09-06 06:57:02 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.01       1.01       0.01            -0.01             0.03                 0.99                 1.03
sex        -0.59       0.55       0.17            -0.92            -0.26                 0.40                 0.77
ph.ecog     0.62       1.85       0.18             0.26             0.98                 1.29                 2.66
ph.karno    0.02       1.02       0.01            -0.00             0.03                 1.00                 1.03
pat.karno  -0.01       0.99       0.01            -0.03             0.00                 0.97                 1.00
wt.loss    -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00

            cmp to     z      p   -log2(p)
covariate                                 
age           0.00  1.35   0.18       2.50
sex           0.00 -3.48 <0.005      10.98
ph.ecog       0.00  3.35 <0.005      10.27
ph.karno      0.00  1.66   0.10       3.37
pat.karno     0.00 -1.78   0.08       3.72
wt.loss       0.00 -1.72   0.08       3.56
---
Concordance = 0.65
Partial AIC = 1463.29
log-likelihood ratio test = 37.68 on 6 df
-log2(p) of ll-ratio test = 19.55

In [25]:
coxph.check_assumptions(data_1, p_value_threshold=0.05)

Proportional hazard assumption looks okay.


[]

### using scikit-survival package

In [ ]:
!pip install -U scikit-survival

## Setting kernel function

In [ ]:
def k_con()